In [2]:
import torch
import glob
from bounding_box import BoundingBox
import cv2

In [3]:
model = torch.hub.load('yolov5', 'custom', 
    path='./yolov5/runs/train/exp30/weights/best.pt', 
    source='local')

YOLOv5 🚀 2023-2-26 Python-3.10.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA A40, 45494MiB)

Fusing layers... 
Model summary: 267 layers, 46124433 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [4]:
img_paths = glob.glob("data/output/val/images/*.png")
img_paths[:5]

['data/output/val/images/7_231.png',
 'data/output/val/images/9_28.png',
 'data/output/val/images/0_115.png',
 'data/output/val/images/6_20.png',
 'data/output/val/images/5_117.png']

In [8]:
def getXY(xy):
    plates = xy[xy['name'].str.contains("Plate")]
    if(len(plates) > 0):
        plates = plates.reset_index()
        plates["xCenter"] = plates['xmin'] + plates['xmax'] / 2
        plates["yCenter"] = plates['ymin'] + plates['ymax'] / 2
        plates["width"] = plates['xmax'] - plates['xmin']
        plates["height"] = plates['ymax'] - plates['ymin']
        return [bbox for bbox in plates.apply(lambda x: BoundingBox(*x[['xCenter', 'yCenter', 'width', 'height']]), axis=1)]
    else:
        return None

In [9]:
bboxes = getXY(model(img_paths[1], size=640).pandas().xyxy[0])
bboxes

[Center: (399.1624450683594px, 189.15540313720703px)
       	Width: 41.681640625px
         Height: 36.69276428222656px
         Initalized at: 05/04/2023 17:26:17:165,
 Center: (305.9253234863281px, 189.3949203491211px)
       	Width: 16.13787841796875px
         Height: 30.336288452148438px
         Initalized at: 05/04/2023 17:26:17:166]

In [18]:
cam_feed = cv2.VideoCapture(0)
cam_feed.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam_feed.set(cv2.CAP_PROP_FRAME_HEIGHT, 750)

[ WARN:0@240.613] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


False

[ WARN:0@240.613] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [23]:
while True:
    # yolo detection
    ret, img = cam_feed.read()   
    pred = model(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), size=640)


    output_img = pred.render()[0]
    # Kalman filter prediction
    plate_loc = getXY(pred.pandas().xyxy[0])
    if(plate_loc is not None):
        x, y = plate_loc

        # xpred, ypred = kf.predict(x, y)

        # draw the circle
        # cv2.circle(output_img, [int(xpred), int(ypred)], 10, color=(0, 165, 255), thickness=3)
        
    # display the frame
    cv2.imshow("", cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR))     
    
    if (cv2.waitKey(1) & 0xFF == ord("q")) or (cv2.waitKey(1)==27):
        break


error: OpenCV(4.6.0) /croot/opencv-suite_1676452025216/work/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
